In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, make_scorer
from sklearn.feature_selection import SelectFromModel
from xgboost import XGBRegressor
from sklearn.impute import SimpleImputer
# Настройка визуализаций
%matplotlib inline

In [2]:
df = pd.read_excel('data/data.xlsx', index_col='Unnamed: 0')

In [3]:
# первичный осмотр данных
print("Первые строки:")
display(df.head())

print('\ninfo:')
df.info()

print("\nОсновные метрики наблюдаемых переменных:")
display(df[['IC50, mM', 'CC50, mM', 'SI']].describe())

# Анализ пропусков
nan_df = df.isnull()
missing = nan_df.sum()
missing = missing[missing > 0]
print('Столбцы с пропущенными значениями: ', ', '.join(missing.index.values))
print('Количество пропущенных значений: ', missing.any())
print('\nСоединения с пропущенными значениями:')
nan_df.loc[nan_df['MaxPartialCharge']==True][missing.index.values]

Первые строки:


,"IC50, mM","CC50, mM",SI,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,6.239374,175.482382,28.125000,5.094096,5.094096,0.387225,0.387225,0.417362,42.928571,384.652,...,0,0,0,0,0,0,0,0,3,0
1,0.771831,5.402819,7.000000,3.961417,3.961417,0.533868,0.533868,0.462473,45.214286,388.684,...,0,0,0,0,0,0,0,0,3,0
2,223.808778,161.142320,0.720000,2.627117,2.627117,0.543231,0.543231,0.260923,42.187500,446.808,...,0,0,0,0,0,0,0,0,3,0
3,1.705624,107.855654,63.235294,5.097360,5.097360,0.390603,0.390603,0.377846,41.862069,398.679,...,0,0,0,0,0,0,0,0,4,0
4,107.131532,139.270991,1.300000,5.150510,5.150510,0.270476,0.270476,0.429038,36.514286,466.713,...,0,0,0,0,0,0,0,0,0,0



info:
<class 'pandas.core.frame.DataFrame'>
Index: 1001 entries, 0 to 1000
Columns: 213 entries, IC50, mM to fr_urea
dtypes: float64(107), int64(106)
memory usage: 1.6 MB

Основные метрики наблюдаемых переменных:


,"IC50, mM","CC50, mM",SI
count,1001.000000,1001.000000,1001.000000
mean,222.805156,589.110728,72.508823
std,402.169734,642.867508,684.482739
min,0.003517,0.700808,0.011489
25%,12.515396,99.999036,1.433333
50%,46.585183,411.039342,3.846154
75%,224.975928,894.089176,16.566667
max,4128.529377,4538.976189,15620.600000


Столбцы с пропущенными значениями:  MaxPartialCharge, MinPartialCharge, MaxAbsPartialCharge, MinAbsPartialCharge, BCUT2D_MWHI, BCUT2D_MWLOW, BCUT2D_CHGHI, BCUT2D_CHGLO, BCUT2D_LOGPHI, BCUT2D_LOGPLOW, BCUT2D_MRHI, BCUT2D_MRLOW
Количество пропущенных значений:  True

Соединения с пропущенными значениями:


,MaxPartialCharge,MinPartialCharge,MaxAbsPartialCharge,MinAbsPartialCharge,BCUT2D_MWHI,BCUT2D_MWLOW,BCUT2D_CHGHI,BCUT2D_CHGLO,BCUT2D_LOGPHI,BCUT2D_LOGPLOW,BCUT2D_MRHI,BCUT2D_MRLOW
78,True,True,True,True,True,True,True,True,True,True,True,True
79,True,True,True,True,True,True,True,True,True,True,True,True
80,True,True,True,True,True,True,True,True,True,True,True,True


In [4]:
# т. к. пустых значений не много, а заполнять их не представляется возможным - удаляем их
df.dropna(how='any', inplace=True)

# Регрессия по IC50

В первую очередь для регрессии необходимо отобрать признаки, которые будут влиять на целевую переменную. Для этого используем градиентный бустинг.

In [5]:
X_ic50 = df.drop(['IC50, mM', 'CC50, mM', 'SI'], axis=1)
y_ic50 = df['IC50, mM']

# берем GBR для выбора признаков
gbr_ic50 = GradientBoostingRegressor(random_state=42)
selector_ic50 = SelectFromModel(gbr_ic50, threshold="median")
selector_ic50.fit(X_ic50, y_ic50)
selected_features_ic50 = X_ic50.columns[selector_ic50.get_support()]
print(f"Всего отобрано признаков: {len(selected_features_ic50)}")

# проверка на мультиколлинеарность
corr_matrix_ic50 = X_ic50[selected_features_ic50].corr().abs()
upper_ic50 = corr_matrix_ic50.where(np.triu(np.ones(corr_matrix_ic50.shape), k=1).astype(bool))
duplicates_ic50 = [column for column in upper_ic50.columns if any(upper_ic50[column] > 0.9)]
final_features_ic50 = [f for f in selected_features_ic50 if f not in duplicates_ic50]
print(f"Всего отобрано признаков после удаления мультиколлинеарности: {len(final_features_ic50)}")

X_final_ic50 = X_ic50[final_features_ic50]
X_train_ic50, X_test_ic50, y_train_ic50, y_test_ic50 = train_test_split(X_final_ic50, y_ic50, test_size=0.2, random_state=42)

Всего отобрано признаков: 105
Всего отобрано признаков после удаления мультиколлинеарности: 81


In [7]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

rf_ic50 = RandomForestRegressor(random_state=42)
grid_search_rf_ic50 = GridSearchCV(
    estimator=rf_ic50,
    param_grid=param_grid,
    scoring='neg_mean_absolute_error',
    cv=5,
    n_jobs=-1
)
grid_search_rf_ic50.fit(X_train_ic50, y_train_ic50)
best_rf = grid_search_rf_ic50.best_estimator_

y_pred_rf_ic50 = best_rf.predict(X_test_ic50)

print("\nМодель случайного леса:")
print(f"Параметры: {grid_search_rf_ic50.best_params_}")
print(f"MAE на тесте: {mean_absolute_error(y_test_ic50, y_pred_rf_ic50):.3f}")
print(f"R² на тесте: {r2_score(y_test_ic50, y_pred_rf_ic50):.3f}")


Модель случайного леса:
Параметры: {'max_depth': 15, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
MAE на тесте: 205.232
R² на тесте: 0.451


In [ ]:
param_grid = {
    'n_estimators': [50, 150, 200],
    'learning_rate': [0.1, 0.15],
    'max_depth': [5, 10],
}

gbr_ic50 = GradientBoostingRegressor(random_state=42)
grid_search_gbr_ic50 = GridSearchCV(
    gbr_ic50, 
    param_grid, 
    cv=5, 
    scoring='neg_mean_squared_error',
    n_jobs=-1)
grid_search_gbr_ic50.fit(X_train_ic50, y_train_ic50)

best_gbr_ic50 = grid_search_gbr_ic50.best_estimator_
y_pred_gbr_ic50 = best_gbr_ic50.predict(X_test_ic50)

print("\nМодель градиентного спуска:")
print(f"Параметры: {grid_search_gbr_ic50.best_params_}")
print(f"MAE на тесте: {mean_absolute_error(y_test_ic50, y_pred_gbr_ic50):.3f}")
print(f"R² на тесте: {r2_score(y_test_ic50, y_pred_gbr_ic50):.3f}")


Модель градиентного спуска:
Параметры: {'learning_rate': 0.15, 'max_depth': 5, 'n_estimators': 50}
MAE на тесте: 116.288
R² на тесте: 0.590


Проанализированно 2 модели и видно, что модель 2 GradientBoostingRegressor лучше (ниже MAE, выше R2).

Параметры: {'learning_rate': 0.15, 'max_depth': 5, 'n_estimators': 50}

MAE на тесте: 116.288

R² на тесте: 0.590